# Lab 2: Códigos Cíclicos

Alunos: Fernando Zanchitta e Guilherme Kowalczuk

# 1. Simulação

In [4]:
from channel import BSC
from encoder import NaiveEncoder, ConvolutionalEncoder
from lab1 import Simulator
from helpers import safe_load

import numpy as np
import random
random.seed(2023)

p = [5e-1, 2e-1, 1e-1, 5e-2, 2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 5e-4, 2e-4, 1e-4, 5e-5, 2e-5, 1e-5]
k = 4
l = 1000000//k

## 1.1. Naive Encoder

In [ ]:
k = 4
l = 1000000//k
naive_encoder = NaiveEncoder()

channels = map(BSC, p)
sim = map(lambda channel: Simulator(naive_encoder, channel, k=k, l=l), channels)
naive_sim = list(map(Simulator.simulate, sim))

print(list(zip(p, naive_sim)))

## 1.2. Convolutional Encoder

In [5]:
n = 3
m = 3
G = np.array([[1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 1]])
conv_encoder = ConvolutionalEncoder(n, m, G)

k = 10000
l = 1

channels = map(BSC, p)
sim = map(lambda channel: Simulator(conv_encoder, channel, k=k, l=l), channels)
conv_sim = list(map(Simulator.simulate, sim))

print(list(zip(p, conv_sim)))

[(0.5, 0.4967), (0.2, 0.1903), (0.1, 0.0076), (0.05, 0.0), (0.02, 0.0), (0.01, 0.0), (0.005, 0.0), (0.002, 0.0), (0.001, 0.0), (0.0005, 0.0), (0.0002, 0.0), (0.0001, 0.0), (5e-05, 0.0), (2e-05, 0.0), (1e-05, 0.0)]


In [ ]:
import matplotlib.pyplot as plt

plt.plot(p, naive_sim, p, conv_sim)